In [1]:
import os, sys
import torch
import argparse

In [2]:
#!export PYTHONPATH=$PYTHONPATH:/data1/practical-sose23/castellvi/castellvi_prediction/bids
sys.path.append('/data1/practical-sose23/castellvi/castellvi_prediction/bids')

In [17]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from utils._prepare_data import DataHandler
from modules.ResNetModule import ResNet
from modules.VerSeDataModule import VerSeDataModule
from modules.DenseNetModule import DenseNet
from modules.DenseNetModule_v2 import DenseNetV2
from utils.environment_settings import env_settings
import torch
from torch import nn

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(env_settings.CUDA_VISIBLE_DEVICES)
if torch.cuda.is_available():
    print('Running on GPU #' + str(torch.cuda.current_device()))

Running on GPU #0


In [12]:
parser = argparse.ArgumentParser(description='Evaluation settings')

parser = argparse.ArgumentParser(description='Evaluation settings')
parser.add_argument('--data_root', nargs='+', default=[str(os.path.join(env_settings.DATA, 'dataset-verse19')),
                                                        str(os.path.join(env_settings.DATA, 'dataset-verse20')),
                                                        str(os.path.join(env_settings.DATA, 'dataset-tri'))])
parser.add_argument('--data_types', nargs='+', default=['rawdata', 'derivatives'])
parser.add_argument('--img_types', nargs='+', default=['ct', 'subreg', 'cortex'])
parser.add_argument('--master_list', default= str(os.path.join(env_settings.ROOT, 'src/dataset/Castellvi_list_Final_Split.xlsx')))
parser.add_argument('--classification_type', default='right_side')
parser.add_argument('--castellvi_classes', nargs='+', default=['1a', '1b', '2a', '2b', '3a', '3b', '4', '0'])
parser.add_argument('--model', default='densenet')
parser.add_argument('--phase', default='train')
parser.add_argument('--scheduler', default='ReduceLROnPlateau')
parser.add_argument('--optimizer', default='AdamW')
parser.add_argument('--learning_rate', type=float, default=0.0001)
parser.add_argument('--weight_decay', type=float, default=0.0001)
parser.add_argument('--total_iterations', type=int, default=100)
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--accumulate_grad_batches', type=int, default=1)
parser.add_argument('--num_workers', type=int, default=8)
parser.add_argument('--save_intervals', type=int, default=10)
parser.add_argument('--n_epochs', type=int, default=100)
parser.add_argument('--resume_path', default='')
parser.add_argument('--experiments', default=env_settings.EXPERIMENTS)
parser.add_argument('--gpu_id', default='0')
parser.add_argument('--n_devices', type=int, default=1)
parser.add_argument('--manual_seed', type=int, default=1)
parser.add_argument('--num_classes', type=int, default=3)
parser.add_argument('--port', type=int, default=2023)
parser.add_argument('--model_type', type=str, default='')
parser.add_argument('--dropout_prob', type=float, default=0.3)


parser.add_argument('--rotate_range', type=int, default=10)
parser.add_argument('--shear_range', type=float, default=0.2)
parser.add_argument('--translate_range', type=float, default=0.15)
parser.add_argument('--scale_range', nargs='+', default=[0.9, 1.1])
parser.add_argument('--aug_prob', type=float, default=0.5)

parser.add_argument('--use_seg', default=True)
parser.add_argument('--no_cuda', default=False)
parser.add_argument('--weighted_sample', default=True)
parser.add_argument('--weighted_loss', default=False)
parser.add_argument('--flip_all', default=True)
parser.add_argument('--cross_validation',default=False)
parser.add_argument('--use_bin_seg', default=True)
parser.add_argument('--use_zero_out',default=True)

parser.add_argument('--elastic_transform', default=True)
parser.add_argument('--seg_comparison', default=False)

parser.add_argument('--version_no', type=int, default=10)
parser.add_argument('--dataset', nargs='+', default=['verse', 'tri'])
parser.add_argument('--eval_type', type=str, default='test')

params, unknown = parser.parse_known_args()


In [23]:
base_experiment = params.experiments + '/baseline_models/' + params.model
ckpt_path = base_experiment + '/best_models/version_' + str(params.version_no)
best_model= os.listdir(ckpt_path)[0]
best_model_path = os.path.join(ckpt_path, best_model)

In [24]:
data_size = (96,78,78) if (params.classification_type == "right_side" or params.classification_type == "right_side_binary") else (128,86,136)
num_classes = params.num_classes
softmax = nn.Softmax(dim=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
def initialize_model(params):
    if params.model == 'densenet':
        model = DenseNet(opt=params, num_classes=params.num_classes, data_size=data_size, data_channel=1)
    elif params.model == 'resnet' or params.model == 'resnetc':
        model = ResNet(opt=params, num_classes=num_classes)
    else:
        raise NotImplementedError("Model not recognized. Implement the model initialization.")
    return model

In [29]:
def load_model(path, params):
    model = initialize_model(params)
    model.load_state_dict(torch.load(path)['state_dict'])
    model.to(device)
    model.eval()
    return model

In [30]:
model = load_model(best_model_path, params)

DenseNet loaded with 3 classes, data_size (96, 78, 78) and 1 channel
